In [1]:
import sqlite3

In [11]:
class Library_DB:
    def __init__(self, Name):
        self.Name = Name
        conn = sqlite3.connect(self.Name)
        c = conn.cursor()
        c.execute(f'''CREATE TABLE IF NOT EXISTS Book
             (id INT PRIMARY KEY NOT NULL,
              Author TEXT,
              Title TEXT, 
              publish_year INTEGER
              )''')
        c.execute(f'''CREATE TABLE IF NOT EXISTS Reader
             (id INT PRIMARY KEY NOT NULL,
              Name TEXT)''')
        c.execute(f'''CREATE TABLE IF NOT EXISTS Record
             (reader_id INT, 
              book_id INT,
              takeing_data TEXT,
              returning_date TEXT,
              FOREIGN KEY (reader_id) REFERENCES Reader (id),
              FOREIGN KEY (book_id) REFERENCES Book (id)
              )''')
        conn.commit()
        conn.close()
        
    def add_book(self, book):
        conn = sqlite3.connect(self.Name)
        c = conn.cursor()
        c.execute(f'INSERT INTO Book VALUES (?, ?, ?, ?);',
                 book)
        conn.commit()
        conn.close()
        
    def add_reader(self, reader):
        conn = sqlite3.connect(self.Name)
        c = conn.cursor()
        c.execute(f'INSERT INTO Reader VALUES (?, ?);',
                 reader)
        conn.commit()
        conn.close()
        
    def print_books(self):
        conn = sqlite3.connect(self.Name)
        c = conn.cursor()
        for row in c.execute(f"SELECT id, Author, Title, publish_year FROM Book"):
            print(row)
        conn.commit()
        conn.close()
        
    def print_readers(self):
        conn = sqlite3.connect(self.Name)
        c = conn.cursor()
        for row in c.execute(f"SELECT id, Name FROM Reader"):
            print(row)
        conn.commit()
        conn.close()
        
    def taken_books(self):
        conn = sqlite3.connect(self.Name)
        c = conn.cursor()
        for row in c.execute(f'''SELECT 
                                reader_id, book_id, 
                                takeing_data, returning_date 
                                FROM Record'''):
            print(row)
        conn.commit()
        conn.close()
        
    def take_book(self, book, ID, date):
        conn = sqlite3.connect(self.Name)
        c = conn.cursor()
        c.execute(f'SELECT * FROM Book WHERE Title =  ?', (book,))
        book_id =c.fetchall() 
        c.execute(f'INSERT INTO Record VALUES(?, ?, ?, ?)', 
                  (ID, book_id[0][0], date, None))
        conn.commit()
        conn.close()
        
    def return_book(self, book, ID, date):
        conn = sqlite3.connect(self.Name)
        c = conn.cursor()
        c.execute(f'''SELECT id, Author, Title, publish_year FROM Book, Record 
                  WHERE Book.Title = ? 
                  and Book.id = Record.book_id 
                  and Record.reader_id = ?
                  and Record.returning_date is NULL ''', (book, ID))
        book_info = c.fetchall()
        c.execute(f'''UPDATE Record SET returning_date = ? 
                  WHERE reader_id = ? and book_id = ? and returning_date is NULL''', 
                  (date, ID, book_info[0][0]))
        conn.commit()
        conn.close()                

### Теперь проверим, что это работает

In [12]:
Lib = Library_DB('Biblio')

In [4]:
book1 = ('1', 'Tolstoy Leo', 'War and Peace',  2010)
Lib.add_book(book1)

book2 = ('2', "Mikhail Bulgakov", "Heart of a Dog", 2000)
Lib.add_book(book2)

reader1 = (1, 'Alex', )
Lib.add_reader(reader1)

In [9]:
Lib.print_books()
Lib.print_readers()

(1, 'Tolstoy Leo', 'War and Peace', 2010)
(2, 'Mikhail Bulgakov', 'Heart of a Dog', 2000)
(1, 'Alex')


In [13]:
Lib.take_book('War and Peace', 1, '2021-05-18')

In [14]:
Lib.taken_books()

(1, 1, '2021-05-18', None)


In [15]:
Lib.return_book('War and Peace', 1, '2021-05-20')

In [16]:
Lib.taken_books()

(1, 1, '2021-05-18', '2021-05-20')


In [17]:
Lib.take_book('War and Peace', 1, '2021-05-18')

In [18]:
Lib.taken_books()

(1, 1, '2021-05-18', '2021-05-20')
(1, 1, '2021-05-18', None)


In [19]:
Lib.return_book('War and Peace', 1, '2021-05-20')

In [20]:
Lib.taken_books()

(1, 1, '2021-05-18', '2021-05-20')
(1, 1, '2021-05-18', '2021-05-20')


In [21]:
Lib.take_book("Heart of a Dog",1, '2021-05-20')


In [22]:
Lib.taken_books()

(1, 1, '2021-05-18', '2021-05-20')
(1, 1, '2021-05-18', '2021-05-20')
(1, 2, '2021-05-20', None)
